In [ ]:
import re
from spellchecker import SpellChecker  # pip install pyspellchecker
from pymongo import MongoClient

# ---------------------------
# 1. Connexion à MongoDB
# ---------------------------
client = MongoClient("mongodb://localhost:27017/")
db = client["ProjetSD"]
collection = db["Laravel"]

# ---------------------------
# 2. Fonctions de prétraitement
# ---------------------------
def clean_text(text):
    """
    Nettoie le texte en éliminant les espaces multiples et les caractères indésirables.
    """
    # Supprime les espaces multiples et sauts de ligne
    text = re.sub(r'\s+', ' ', text)
    # Conserve lettres, chiffres, accents et ponctuation (.,;!?)
    text = re.sub(r'[^a-zA-Z0-9À-ÖØ-öø-ÿ\s.,;!?]', '', text)
    return text.strip()

def segment_text(text):
    """
    Segmente le texte en chapitres ou sections.
    Ici, on suppose que le mot "Chapitre" marque le début d'un nouveau segment.
    """
    segments = text.split("Chapitre")
    # Réintroduit "Chapitre" au début de chaque segment et élimine les segments vides
    segments = [("Chapitre " + seg).strip() for seg in segments if seg.strip()]
    return segments

def correct_text(text):
    """
    Corrige l'orthographe du texte en utilisant pyspellchecker pour le français.
    Si la correction renvoie None pour un mot, le mot original est conservé.
    """
    spell = SpellChecker(language='fr')
    words = text.split()
    corrected_words = []
    for word in words:
        corrected = spell.correction(word)
        # Si aucune correction n'est trouvée, conserver le mot original
        if corrected is None:
            corrected = word
        corrected_words.append(corrected)
    return " ".join(corrected_words)

# ---------------------------
# 3. Parcours et mise à jour des documents
# ---------------------------
for doc in collection.find():
    raw_text = doc.get("content", "")
    if raw_text:
        # Nettoyage du texte
        cleaned = clean_text(raw_text)
        # Segmentation en chapitres/sections
        segments = segment_text(cleaned)
        # Correction orthographique (appliquée ici sur le texte nettoyé)
        corrected = correct_text(cleaned)
        
        # Mise à jour du document dans MongoDB avec les nouveaux champs
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {
                "cleaned_text": cleaned,
                "segments": segments,
                "corrected_text": corrected
            }}
        )
        print(f"Document {doc['_id']} mis à jour avec les champs prétraités.")
    else:
        print(f"Document {doc['_id']} : aucun contenu à traiter.")
